In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import file with username
private = pd.read_csv('e:/PythonProjects/api.csv')
val = private.un[private.api == 'geopy'][0]

In [ ]:
# you will need a free geopy account
geolocator = Nominatim(user_agent=val)

In [ ]:
def gets_temperature_forecast(city_state: str):

    location = geolocator.geocode(city_state)
    lat = location.latitude
    lon = location.longitude
    
    print(lat, lon)
    
    # https://www.weather.gov/documentation/services-web-api
    res = requests.get(f'https://api.weather.gov/points/{lat},{lon}')
    status = res.status_code
    
    if status == 200:
        points = res.json()

        gridId = points['properties']['gridId']
        gridX = points['properties']['gridX']
        gridY = points['properties']['gridY']

        temp_api = requests.get(f'https://api.weather.gov/gridpoints/{gridId}/{gridX},{gridY}').json()

        data = temp_api['properties']['temperature']['values']

        df = pd.DataFrame(data)
        df.validTime = df.validTime.str.split('/', expand=True)[0]
        df.validTime = pd.to_datetime(df.validTime)
        df.rename(columns={'value': '°C'}, inplace=True)
        df['°C'] = df['°C'].round(2)
        df['°F'] = (df['°C'] * 9/5 + 32).round(1)
        df['date'] = df.validTime.dt.date
        df['time'] = df.validTime.dt.time
        
        print('Finished')

    #     for _, (_, deg_c, deg_f, date, time) in df.iterrows():
    #         print(f'Date: {date} Time: {time} Temperature: {deg_c:0.2f} °C ({deg_f:0.1f} °F)')
    else:
        print(f'Invalid Staus Code: {status}')
        df = None
        
    return df

In [ ]:
# city_state = 'Portland, OR'
city_state = input('Input a city and state in the form (City, ST): ')
d = gets_temperature_forecast(city_state)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
p1 = sns.scatterplot(data=d, x='validTime', y='°C', ax=ax)
p1.set_xlabel('Date')
p1.set_title(f'Temperature: {city_state}')
p1.grid()